## Research Track II Assignment
## Pablo Moreno Moreno - S5646698

This is the Jupyter Notebook created to interact with the simulation of the second assignment of Research Track I.

### Import libraries

In [ ]:
import rospy
from geometry_msgs.msg import PoseStamped
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from my_assignment_2.msg import PosVel
from assignment_2_2022.msg import PlanningActionResult

# Bring in the SimpleActionClient
import actionlib 
# Bring in the PlanningAction type, composed by the types PlanningGoal, PlanningResult and PlanningFeedback
from assignment_2_2022.msg import PlanningAction, PlanningGoal

import ipywidgets as widgets
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
import matplotlib.lines as mlines
from matplotlib.ticker import MaxNLocator

### Widget definition

In [ ]:
%matplotlib widget

# Send button
send_button = widgets.Button(
    description='Send',
    disabled=False,
    button_style='success',
    layout = widgets.Layout(width='350px')
)

# Cancel button
cancel_button = widgets.Button(
    description='Cancel',
    disabled=False,
    button_style='danger',
    layout = widgets.Layout(width='350px')
)

# Widget for the status of the robot
status_text = widgets.Text(value='Waiting for the user to enter a goal',
                           description='Status:',
                           disabled=True,
                           layout = widgets.Layout(width='350px'))

# Widget for the distance to the closest obstacle
box_distance = widgets.FloatText(value=-1,
                        description="Distance to the closest obstacle:", 
                        style= {'description_width': 'initial'}, 
                        disabled = True,
                        layout = widgets.Layout(width='350px'))

# Widget for the number of goals reached
box_reached = widgets.IntText(value=0, disabled=True)
def update_box_reached(change):
    plt.figure(2)
    plt.bar(["Reached", "Cancelled"], [n_goals_reached, n_goals_cancelled], align='center', width=0.8, color=['green', 'red'])
    plt.show()
box_reached.observe(update_box_reached, 'value')

# Widget for the number of goals cancelled
box_cancelled = widgets.IntText(value=0, disabled=True)
def update_box_cancelled(change):
    plt.figure(2)
    plt.bar(["Reached", "Cancelled"], [n_goals_reached, n_goals_cancelled], align='center', width=0.8, color=['green', 'red'])
box_cancelled.observe(update_box_cancelled, 'value')

### Class to visualize the robot's position using *FuncAnimation*

In [ ]:
class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro', markersize=3)
        self.x_data, self.y_data = [] , []
        
        self.ax.set_xlabel('x') # Add an x-label to the axes
        self.ax.set_ylabel('y') # Add a y-label to the axes
        self.ax.set_title("Robot's position and goals' positions in the environment") # Add a title to the axes
        plt.grid() # Show grid
        
        # Legend
        robot_position = mlines.Line2D([], [], linewidth = 3, color='red', label='Robot position')
        current_goal = mlines.Line2D([], [], linewidth = 0, color='blue', marker='*', label='Current goal') 
        reached_goals = mlines.Line2D([], [], linewidth = 0, color='green', marker='H', label='Reached goals') 
        cancelled_goals = mlines.Line2D([], [], linewidth = 0, color='orange', marker='H', label='Cancelled goals') 
        plt.legend(handles=[robot_position, current_goal, reached_goals, cancelled_goals])
        
    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln
    
    def odom_callback(self, msg):
        
        my_pos_vel = PosVel() # This is the msg we'll publish
    
        # Fill the four fields relying on the data received via the topic /odom
        my_pos_vel.x = msg.pose.pose.position.x
        my_pos_vel.y = msg.pose.pose.position.y
        my_pos_vel.vel_x = msg.twist.twist.linear.x
        my_pos_vel.vel_z = msg.twist.twist.angular.z

        pub.publish(my_pos_vel) # Publish
        
        # Update the arrays containing the position of the robot
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
        
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln  

### Callback function for the laser scanner

In [ ]:
def callback_scan(msg):
    
    # Initialize the minimum distance
    min_val = msg.range_max
    
    # Iterate through the entire array to find the minimum value
    for i in range(len(msg.ranges)):
        if (msg.ranges[i] < min_val) and (msg.ranges[i] >= msg.range_min) and (msg.ranges[i] <= msg.range_max):
            min_val = msg.ranges[i]
    
    # Update the widget corresponding to the distance to the closest obstacle
    global box_distance
    box_distance.value = min_val

### If the goal has been successfully reached...

In [ ]:
def callback_result(result):
    if result.status.status == 3:

        # Update the status
        status_text.value = "Goal reached"
        
        # Increment the parameter related to the number of goals reached
        global n_goals_reached
        n_goals_reached += 1
        rospy.set_param('goals_reached', n_goals_reached)
        
        # Update the bar graph
        global box_reached
        box_reached.value += 1
        
        # Add a marker to the plot of the environment
        plt.figure(vis.fig.number)
        plt.plot(x.value, y.value, color='green', marker='H')

### If the user clicks the *send*  button...

In [ ]:
def on_send_button_clicked(b):
        
    # Create a goal to send to the action server
    my_goal_pose = PoseStamped()
    my_goal_pose.pose.position.x = x.value
    my_goal_pose.pose.position.y = y.value
    goal = PlanningGoal(target_pose = my_goal_pose)

    # Send the goal to the action server
    client.send_goal(goal)
    
    # Update the status of the robot
    status_text.value = "Heading to the goal (" + str(x.value) + ", " + str(y.value) + ")"
    
    # Add a marker to the plot of the environment
    plt.figure(vis.fig.number)
    plt.plot(x.value, y.value, color='blue', marker='*')

# Associate this callback function to the send button
send_button.on_click(on_send_button_clicked)

### If the user clicks the *cancel*  button...

In [ ]:
def on_cancel_button_clicked(b):
    
    # Cancel the current goal
    client.cancel_goal()
    
    # Update the status
    status_text.value = "Goal cancelled"
    
    # Increment the parameter related to the number of goals cancelled
    global n_goals_cancelled
    n_goals_cancelled += 1
    rospy.set_param('goals_cancelled', n_goals_cancelled)
    
    # Update the bar graph
    global box_cancelled
    box_cancelled.value += 1
    
    # Add a marker to the plot of the environment
    plt.figure(vis.fig.number)
    plt.plot(x.value, y.value, color='orange', marker='H')

# Associate this callback function to the cancel button
cancel_button.on_click(on_cancel_button_clicked)

### Get a new target from user input & Data Visualization

In [ ]:
print("\nTo send a new goal, specify the x and y coordinates and click \'Send\':\n")
x = widgets.FloatText(description='x = ', layout = widgets.Layout(width='350px'))
display(x)
y = widgets.FloatText(description='y = ', layout = widgets.Layout(width='350px'))
display(y)

# Display send button
output = widgets.Output()
display(send_button, output)

# Display cancel button
print("To cancel the current goal, click \'Cancel\':\n")
display(cancel_button, output)

# Display the status and distance boxes
display(status_text)
display(box_distance)

# Initialize a rospy node so that the SimpleActionClient can publish and subscribe over ROS
rospy.init_node('planning_client')

# Create an instance of the class Visualiser
vis = Visualiser()

# Define the publisher fot the topic /pos_vel
pub = rospy.Publisher("/pos_vel", PosVel, queue_size=10)

# Subscribe to the topics /odom, /reaching_goal/result, and /scan
sub1 = rospy.Subscriber("/odom", Odometry, vis.odom_callback)
sub2 = rospy.Subscriber("/reaching_goal/result", PlanningActionResult, callback_result)
sub3 = rospy.Subscriber("/scan", LaserScan, callback_scan)

# FuncAnimation
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.show(block=False)

# Counters for the number of goals reached and cancelled, in total
n_goals_reached = 0
n_goals_cancelled = 0

# Initialize the parameters, in the ROS parameter service, related to these counters
rospy.set_param('goals_reached', n_goals_reached)
rospy.set_param('goals_cancelled', n_goals_cancelled)

# Create the SimpleActionClient. The service is called '/reaching_goal' and the type of action is 'PlanningAction'
client = actionlib.SimpleActionClient('reaching_goal', PlanningAction)

# Wait until the action server has started up and started listening for goals
client.wait_for_server()

# Graph for the number of goals reached and cancelled
ax = plt.figure(2).gca()
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
plt.bar(["Reached", "Cancelled"], [n_goals_reached, n_goals_cancelled], align='center', width=0.8, color=['green', 'red'])
plt.title("Number of goals reached and cancelled")
plt.grid(axis='y')
plt.show()